In [27]:
#https://query2.finance.yahoo.com/ws/fundamentals-timeseries/v1/finance/timeseries/AAPL?symbol=AAPL"
#https://query2.finance.yahoo.com/v8/finance/chart/AAPL?period1=1704145844&period2=1723236317&interval=1mo

In [1]:
import sys
import os
import pandas as pd

cwd = os.getcwd()

sys.path.append(cwd.replace('notebooks','')+'code')

In [5]:
from source.scraper import get_StockPrices

df = get_StockPrices('NVDA', startSelect='2002-01-02', endSelect='2022-01-02')
df

,adjClose,ticker,quarter_meanADJclose,month_meanADJclose,yearMonthDay
0,0.514411,NVDA,0.448435,0.501850,20020102
1,0.548119,NVDA,0.448435,0.501850,20020103
2,0.532145,NVDA,0.448435,0.501850,20020104
3,0.517851,NVDA,0.448435,0.501850,20020107
4,0.523966,NVDA,0.448435,0.501850,20020108
...,...,...,...,...,...
5031,30.898392,NVDA,27.471358,29.885646,20211227
5032,30.276327,NVDA,27.471358,29.885646,20211228
5033,29.955814,NVDA,27.471358,29.885646,20211229
5034,29.541439,NVDA,27.471358,29.885646,20211230


In [ ]:
from source.scraper import generate_CIK_TICKER


df = generate_CIK_TICKER()

In [2]:
from spark_deltalake.spark_operations import sparkDelta


spark = sparkDelta()

hadoop-azure is available
azure-storage is available


In [3]:
pydf = spark.get_spark_dataframe('SEC_filings', True)
pydf.show()

+--------------------+----------+----------+--------------------+---+----+----------+----------+--------------+-------+------+------------+--------------------+---------------------+--------------------+------------+------+
|     accessionNumber|filingDate|reportDate|  acceptanceDateTime|act|form|fileNumber|filmNumber|         items|   size|isXBRL|isInlineXBRL|     primaryDocument|primaryDocDescription|             fileURL|yearMonthDay|ticker|
+--------------------+----------+----------+--------------------+---+----+----------+----------+--------------+-------+------+------------+--------------------+---------------------+--------------------+------------+------+
|0001467373-24-000176|2024-05-17|2024-05-15|2024-05-17T09:11:...|   |   4|          |          |              |   4834|     0|           0|xslF345X05/wk-for...|               FORM 4|https://www.sec.g...|    20240515|   ACN|
|0001467373-24-000175|2024-05-17|2024-05-15|2024-05-17T09:11:...|   |   4|          |          |        

In [5]:
pydf = spark.get_spark_dataframe('company_facts', True)
#delete diffDate, endFormat, startFormat
pydf.show()

+--------------------+--------------------+--------------------+----------+----------+--------------------+------+---+----+----------+---------+--------------------+----------+--------------------+--------+-----------+------------+
|             finType|               label|         description|       end|       val|                accn|    fy| fp|form|     filed|    frame|           endFormat|     start|         startFormat|diffDate|monthWindow|yearMonthDay|
+--------------------+--------------------+--------------------+----------+----------+--------------------+------+---+----+----------+---------+--------------------+----------+--------------------+--------+-----------+------------+
|IncreaseDecreaseI...|Increase (Decreas...|The increase (dec...|2020-09-30|    -4.8E7|0000066740-21-000013|2021.0| Q3|10-Q|2021-10-26|      NaN|java.util.Gregori...|2020-01-01|java.util.Gregori...|   273.0|        9.0|    20200930|
|IncreaseDecreaseI...|Increase (Decreas...|The increase (dec...|2021-09-

In [7]:
spark.spark.sql('SELECT endFormat FROM company_facts').show()

+--------------------+
|           endFormat|
+--------------------+
|java.util.Gregori...|
|java.util.Gregori...|
|java.util.Gregori...|
|java.util.Gregori...|
|java.util.Gregori...|
|java.util.Gregori...|
|java.util.Gregori...|
|java.util.Gregori...|
|java.util.Gregori...|
|java.util.Gregori...|
|java.util.Gregori...|
|java.util.Gregori...|
|java.util.Gregori...|
|java.util.Gregori...|
|java.util.Gregori...|
|java.util.Gregori...|
|java.util.Gregori...|
|java.util.Gregori...|
|java.util.Gregori...|
|java.util.Gregori...|
+--------------------+
only showing top 20 rows



In [ ]:
df = spark.spark.sql('SELECT DISTINCT ticker FROM SEC_filings').toPandas()
tickers = list(df.values[:,0])

['AOS', 'MMM']

In [ ]:
ticker = 'AOS'

df = spark.get_min_max_fillings_date(ticker)

In [ ]:
spark.sparkStop()

In [ ]:
from source.scraper import get_spy500_formWiki


wiki = get_spy500_formWiki()

In [ ]:
get_companyfacts('CIK0000320193')

In [ ]:
from source.scraper import get_companyfacts
#https://data.sec.gov/api/xbrl/companyfacts/CIK0000320193.json
mergedDF = get_companyfacts('CIK0001045810') #2016-07-31

#mergedDF['fy'] = mergedDF['fy'].replace('',"")
#mergedDF['fy'] = mergedDF['fy'].astype(str).str[:4]
#mergedDF['fy'] = mergedDF['fy'].replace(-1, None)

mergedDF

In [ ]:
"""
https://www.sec.gov/ix?doc=/Archives/edgar/data/0000320193/000114036124024352/ef20028273_8k.htm


https://data.sec.gov/api/xbrl/companyconcept/CIK0000320193/us-gaap/AccountsPayableCurrent.json - info about reports

https://data.sec.gov/api/xbrl/companyfacts/CIK0000320193.json

/0001140361|24|024352/
accessionNumber - 0001140361-24-024352

"""

In [ ]:
from source.scraper import generate_CIK_TICKER

sec_ticker = generate_CIK_TICKER()
sec_ticker

In [ ]:
from source.scraper import get_SEC_filings


ticker = 'AAPL'

filings = get_SEC_filings('CIK0000320193', 'AAPL')

filings